In [4]:
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.animation as animation
#from IPython.display import HTML
#from skimage import color
from scipy.ndimage.filters import convolve
import moviepy.editor as mpy
import matplotlib.image as mpimg
from moviepy.editor import ImageSequenceClip
%matplotlib inline


def infection_propagation(SIR, infection_rate, incubation_rate):
    S,I,R = SIR
    newly_infected = infection_rate*R*S
    newly_rampaging = incubation_rate*I
    dS = - newly_infected
    dI = newly_infected - newly_rampaging
    dR = newly_rampaging
    assembled=np.array([dS, dI, dR])
    return assembled

def disperse_virus(SIR, dispersion_kernel, dispersion_rates):
    dispersed= np.array( [convolve(e, dispersion_kernel, cval=0)*r
                       for (e,r) in zip(SIR, dispersion_rates)])
    return dispersed



In [7]:

infection_rate = 0.5
incubation_rate = 0.2

dispersion_kernel = np.array([[0.5, 1 , 0.5],
                                [1  , -6, 1],
                                [0.5, 1, 0.5]]) 
dispersion_rates  = [0, 0.05, 0.03]

population_map = mpimg.imread('swiss_dot_map.png')
swiss_map=population_map[:, :, 2]
pop_map=(1-swiss_map)

SIR = np.zeros((3, np.shape(population_map)[0], np.shape(population_map)[1]),  dtype=float)
SIR[0] = pop_map

start = (200, 600) #Patient 0 position

SIR[1,start[0], start[1]] = 0.8 # 

dt = 1.0 
country = {'SIR':SIR, 't':0}
coefs = np.array([2,25,25]).reshape((3,1,1))

ims = []

#Generate list of images for video simmulation
for i in range(600):
    
    infect = infection_propagation(country['SIR'], infection_rate, incubation_rate)
    disperse = disperse_virus(country['SIR'], dispersion_kernel, dispersion_rates)
    country['SIR'] += dt*(infect+disperse)
    country['t']= country['t']+dt

    accentuated = 255*coefs*country['SIR']
    image = accentuated[::-1].swapaxes(0,2).swapaxes(0,1)
    
    frame1=np.minimum(255, image)
    ims.append(frame1)
    



In [ ]:
#Write movie
#clip = ImageSequenceClip(list(array), fps=20)
clip = ImageSequenceClip(ims, fps=25)
#clip.write_gif('simmulation.gif', fps=15)
clip.write_videofile('simmulation.mp4', fps=25)